In [1]:
import pandas as pd
import numpy as np
import os
import re
import json as js
from pathlib import Path
from tqdm import tqdm
from ast import literal_eval

In [2]:
directory_kamus = "Daftar Kamus Analisis Machine Readable"
directory_kamus_full = "[Full] Daftar Kamus Ekstraksi"

### Algoritma One Entry Corpus ###

In [3]:
# Algoritma Tambahan
POS = ["v","a","n","pron","adv","num","p"]

def is_contain_bold_and_italic(font):
    contains_bold = False; contains_italic = False
    for i in font:
        if "bold" in i.lower(): contains_bold = True
        if "italic" in i.lower(): contains_italic = True
        if contains_bold == True and contains_italic == True: return True
    return False

def is_last_fonem(s): # baru dapat handle fonem (/../) dan ([...])
    if re.match(r'^.*\]$',str(s)): return True
    if re.match(r'^.*\/$',str(s)): return True
    return False

def is_start_fonem(s): # baru dapat handle fonem (/../) dan ([...])
    if re.match(r'^\[.*',str(s)): return True
    if re.match(r'^\/.*',str(s)): return True
    return False

def is_bold_contains_POS(s):
    kata = s.strip()
    
    if len(kata) > 1:
        if is_contain_only_whitespaces(kata[-2]) and (kata[-1] in POS): return True
    else:
        if (kata[-1] in POS): return True
    
    return False

def is_contain_only_whitespaces(s):
    if re.match(r'^\s*$', str(s)): return True
    return False

def is_end_entri(s):
    symbol = [";",",",":"]
    if s in symbol:
        return True
    else:
        return False

In [4]:
# Algoritma join seperate entry
def join_seperate_entry(data):
    result_entry = []
    result_entry_with_font_size_pos = []
    result_posisi = []
    result_page = []
    is_padanan_lema = []
    
    i = 0
    while i < len(data["Entri"]):
        if i == (len(entries)-1): 
            result_entry.append(data["Entri"][i])
            result_entry_with_font_size_pos.append(data["entry_font_size_pos"][i])
            result_posisi.append(data["posisi_entry"][i])
            result_page.append(data["page"][i])
            is_padanan_lema.append(data["is_padanan_lema"][i])
            break
        
        joined_entry = data["Entri"][i]
        joined_entry_with_font_size_pos = data["entry_font_size_pos"][i]
        curr_posisi = data["posisi_entry"][i]
        curr_page = data["page"][i]
        curr_pl = data["is_padanan_lema"][i]
        
        curr_y0 = joined_entry_with_font_size_pos[-1][-1][1]
        batas_atas = curr_y0 + (curr_y0*(1/100)) # error 1%
        batas_bawah = curr_y0 - (curr_y0*(1/100)) # error 1%
        
        
        nxt= i+1
        if nxt > len(entries)-1: 
            result_entry.append(joined_entry)
            result_entry_with_font_size_pos.append(joined_entry_with_font_size_pos)
            result_posisi.append(curr_posisi)
            result_page.append(curr_page)
            is_padanan_lema.append(curr_pl)
            break
            
        else:
            nxt_y0 = round(data["entry_font_size_pos"][nxt][0][-1][1],2)
            while batas_bawah <= nxt_y0 <= batas_atas:
                if curr_pl == 1: break

                joined_entry = joined_entry + " " + data["Entri"][nxt]
                joined_entry_with_font_size_pos.extend(data["entry_font_size_pos"][nxt])

                if isinstance(data["page"][nxt],list):
                    curr_page = data["page"][nxt]
                elif isinstance(curr_page,list):
                    curr_page = curr_page
                elif curr_page != page[nxt]:
                    curr_page = [curr_page, data["page"][nxt]]

                nxt += 1
                
                if nxt == (len(entries)): break
                    
                nxt_y0 = round(data["entry_font_size_pos"][nxt][0][-1][1],2)

                curr_y0 = joined_entry_with_font_size_pos[-1][-1][1]
                curr_pl = data["is_padanan_lema"][nxt]
                batas_atas = curr_y0 + (curr_y0*(1/100)) # error 1%
                batas_bawah = curr_y0 - (curr_y0*(1/100)) # error 1%

            result_entry.append(joined_entry)
            result_entry_with_font_size_pos.append(joined_entry_with_font_size_pos)
            result_posisi.append(curr_posisi)
            result_page.append(curr_page)
            is_padanan_lema.append(curr_pl)
            i = nxt
            
    return {
        "Entri":result_entry,
        "entry_font_size_pos":result_entry_with_font_size_pos,
        "posisi_entry":result_posisi,
        "page":result_page,
        "is_padanan_lema":is_padanan_lema
    }

In [5]:
# kategorisasi entry, untuk memisahkan mana yang main entry dan sub entry
def categorize_main_entry(posisi, pages, lema):
    output = []
    
    i = 0; j = 0
    while i < len(pages):
        if isinstance(pages[i], list): # kasus entry cross page
            prev_posisi_x0 = posisi[i-1][0]
            
            if abs(posisi[i][0] - prev_posisi_x0) <= 3:
                output.append(output[i-1])
                
            else:
                batas_atas = round(prev_posisi_x0 + (prev_posisi_x0 * (2/100)),2) # error 2%
                batas_kolom = 2*batas_atas
                
                if posisi[i][0] > batas_atas and posisi[i][0] < batas_kolom:
                    output.append(0)
                    
                else:
                    output.append(1)
                    
            i += 1; j += 1
        
        else:   
            posisi_by_page = []; lema_by_page = []; curr_page = pages[j] 
        
            while curr_page == pages[i]: # kelompokkan entri berdasarkan halaman
                posisi_by_page.append(posisi[j][0])
                lema_by_page.append(lema[j])
                
                j += 1
                
                if j > len(pages) - 1: break
                    
                curr_page = pages[j]
                
            sorted_posisi = sorted(posisi_by_page) # urutkan
            
            i = j; k = 0; l = 0 # update nilai i
            while k < len(posisi_by_page):
                if lema_by_page[k] == 1:
                    if k == len(posisi_by_page)-1:
                        output.append(1); break
                    else:
                        output.append(1)
                        output.append(0)
                        k += 2
                else:
                    if abs(posisi_by_page[k] - sorted_posisi[l]) > 3:
                        output.append(0); k += 1 # jika tidak sesuai urutan

                    else:
                        output.append(1) # index pertama setelah header atau nomor halaman
                        batas_atas = round(posisi_by_page[k] + (posisi_by_page[k] * (2/100)),2) # error 2%
                        batas_kolom = 2*batas_atas

                        m = k + 1
                        if m > len(posisi_by_page) - 1: break

                        nxt_posisi = posisi_by_page[m]
                        while nxt_posisi > batas_atas and nxt_posisi < batas_kolom:
                            output.append(0); m += 1

                            if m > len(posisi_by_page) - 1: 
                                break 

                            nxt_posisi = posisi_by_page[m]

                        k = m
                        if nxt_posisi < batas_kolom:
                            l += 1
                        else:
                            l = m
                
                
    return output 

In [217]:
# pisahkan main entry atau entry-entry pokok yang masih tergabung
def seperate_joined_entry(data, kategori):
    result_entries = []
    result_entries_with_font_size_pos = []
    result_posisi = []
    result_page = []
    
    entry = []
    entry_with_font_size_pos = []
    posisi_dummy = None;
    
    i = 0
    while i < len(data["Entri"]):
        
        if len(data["entry_font_size_pos"][i]) < 2: # jika hanya terdiri dari 1 kata atau 0 kata
            result_entries.append(data["Entri"][i])
            result_entries_with_font_size_pos.append(data["entry_font_size_pos"][i])
            result_posisi.append(data["posisi_entry"][i])
            result_page.append(data["page"][i])
            entry = []; entry_with_font_size_pos = []; posisi_dummy = None
        
        else:
            posisi = data["posisi_entry"][i]
            batas_atas = round(posisi[0] + (posisi[0] * (1/100)),2)

            detail = data["entry_font_size_pos"][i][0] # handle index 0

            entry.append(detail[0].strip())
            entry_with_font_size_pos.append(detail)
            posisi_dummy = detail[-1]

            if (kategori[i] == 0): 
                result_entries.append(data["Entri"][i])
                result_entries_with_font_size_pos.append(data["entry_font_size_pos"][i])
                result_posisi.append(data["posisi_entry"][i])
                result_page.append(data["page"][i])
                entry = []; entry_with_font_size_pos = []; posisi_dummy = None

            else: # pisahkan entri pokok yang tergabung
                batas_bawah = round(posisi[0] - (posisi[0] * (1/100)),2) # error 1%

                for j in range(1,len(data["entry_font_size_pos"][i])):
                    detail = data["entry_font_size_pos"][i][j]; 
                    posisi_x0 = round(float(detail[-1][0]))

                    if batas_bawah <= posisi_x0 <= batas_atas: # pisahkan entry
                        joined_entry = (" ").join(entry)
                        result_entries.append(joined_entry)
                        result_entries_with_font_size_pos.append(entry_with_font_size_pos)
                        result_posisi.append(posisi_dummy)
                        result_page.append(data["page"][i])

                        entry = []; entry_with_font_size_pos = []
                        entry.append(detail[0].strip())
                        entry_with_font_size_pos.append(detail)
                        posisi_dummy = detail[-1]

                    else:
                        entry.append(detail[0].strip())
                        entry_with_font_size_pos.append(detail)
        
        if entry != []:
            joined_entry = (" ").join(entry)
            result_entries.append(joined_entry)
            result_entries_with_font_size_pos.append(entry_with_font_size_pos)
            result_posisi.append(posisi_dummy)
            result_page.append(data["page"][i])
            entry = []; entry_with_font_size_pos = []; posisi_dummy = None

        i += 1
    
    return {
        "Entri":result_entries,
        "entry_font_size_pos":result_entries_with_font_size_pos,
        "posisi_entry":result_posisi,
        "page":result_page
    }

In [219]:
# memisahkan prakategorial
def seperate_prakategorial(data):
    result = {
        "Entri":[],
        "entry_font_size_pos":[],
        "posisi_entry":[],
        "page":[]
    }
    
    for i in range(len(data["Entri"])):
        txt = data["Entri"][i]
        split_txt = txt.strip().split(",",1)
        
        if len(split_txt) < 2 or txt[-1] == ",": # tidak terdapat koma atau koma berada di akhir
            result['Entri'].append(txt)
            result['entry_font_size_pos'].append(data['entry_font_size_pos'][i])
            result['page'].append(data['page'][i])
            result['posisi_entry'].append(data['posisi_entry'][i])
        
        else:
            frst_entri = split_txt[0].strip().split(" ")
            sec_entri = split_txt[1].strip().split(" ")
            
            for j in range(len(frst_entri)):
                frst_entri[j] = frst_entri[j].strip()
            
            for k in range(len(sec_entri)):
                sec_entri[k] = sec_entri[k].strip()
                
            if len(frst_entri) <= 2 and (frst_entri[0] == "" or frst_entri[0] == ","): # koma berada di awal entri
                result['Entri'].append(txt)
                result['entry_font_size_pos'].append(data['entry_font_size_pos'][i])
                result['page'].append(data['page'][i])
                result['posisi_entry'].append(data['posisi_entry'][i])
            
            else:
                inf_frst_entri = data['entry_font_size_pos'][i][:len(frst_entri)]
                
                if "bold" in inf_frst_entri[-1][1].lower() or frst_entri[-1] in POS:
                    if (len(frst_entri) + len(sec_entri)) == len(data['entry_font_size_pos'][i]): # kasus koma menempel
                        frst_entri[-1] = frst_entri[-1] + ","
                        inf_sec_entri = data['entry_font_size_pos'][i][len(frst_entri):]

                    else: # kasus koma tidak menempel
                        frst_entri.append(",")
                        inf_frst_entri = data['entry_font_size_pos'][i][:len(frst_entri)+1]
                        inf_sec_entri = data['entry_font_size_pos'][i][len(frst_entri)+1:]
                        
                    # entri pertama
                    result['Entri'].append(" ".join(frst_entri))
                    result['entry_font_size_pos'].append(inf_frst_entri)
                    result['page'].append(data['page'][i])
                    result['posisi_entry'].append(data['posisi_entry'][i])

                    # entri kedua
                    result['Entri'].append(" ".join(sec_entri))
                    result['entry_font_size_pos'].append(inf_sec_entri)
                    result['page'].append(data['page'][i])
                    result['posisi_entry'].append(data['posisi_entry'][i])

                else: # pemisahan koma tidak valid
                    result['Entri'].append(txt)
                    result['entry_font_size_pos'].append(data['entry_font_size_pos'][i])
                    result['page'].append(data['page'][i])
                    result['posisi_entry'].append(data['posisi_entry'][i])
                
                
    return result

In [220]:
# algoritma bersihkan entry dari fonem
def clean_entry(data):
    result = {
        "Entri":[],
        "entry_font_size_pos":[],
        "posisi_entry":[],
        "page":[]
    }
    
    for i in range(len(data["Entri"])): # remove fonem
        txt = data["Entri"][i] # data text
        
        if not is_contain_only_whitespaces(txt):
            
            entry_font_size_pos = data["entry_font_size_pos"][i]
            txt = re.sub(r'\[.*?\]',"",txt)
            entry_font_size_pos = clean_entry_font_size_paranthesis(entry_font_size_pos)

            txt = re.sub(r'\/.*?\/',"",txt)
            entry_font_size_pos = clean_entry_font_size_slash(entry_font_size_pos)

            clean = re.sub(' +', ' ', txt) ## remove multiple whitespace
            result["Entri"].append(clean.strip())
            result["entry_font_size_pos"].append(entry_font_size_pos)

            result['posisi_entry'].append(data['posisi_entry'][i])
            result['page'].append(data['page'][i])
    
    for j in range(1,len(result['Entri'])): # fix symbol
        array_simbol = []; array_simbol_font_size_pos = []
        
        prev_txt_split = result["Entri"][j-1].split(" ")
        prev_entri_font_size_pos = result['entry_font_size_pos'][j-1]
        
        # buang seluruh simbol, kecuali ; pada entri sebelumnya
        while (prev_txt_split[-1] != "") and (not is_end_entri(prev_txt_split[-1][-1])):
            if (prev_txt_split[-1][0].isalnum()) or (prev_txt_split[-1][-1].isalnum()): 
                break
                
            else:
                if (prev_txt_split==[] or prev_entri_font_size_pos == []):break
                
                array_simbol.append(prev_txt_split[-1])
                array_simbol_font_size_pos.append(prev_entri_font_size_pos[-1])
                del prev_txt_split[-1]
                del prev_entri_font_size_pos[-1]
                
                result["Entri"][j-1] = " ".join(prev_txt_split)
                result['entry_font_size_pos'][j-1] = prev_entri_font_size_pos
            
            if (prev_txt_split==[] or prev_entri_font_size_pos == []):break
        
        txt_split = result['Entri'][j].split(" ")
        if is_end_entri(txt_split[0]): 
            result['Entri'][j-1] = result['Entri'][j-1] + txt_split[0]
            result['entry_font_size_pos'][j-1].append(result['entry_font_size_pos'][j][0])
            
            del txt_split[0]
            result['entry_font_size_pos'][j] = result['entry_font_size_pos'][j][1:]
            result['Entri'][j] = " ".join(txt_split)
        
        if array_simbol != []:
            new_entry = []
            new_entry.extend(array_simbol)
            new_entry.extend(txt_split)
            result['Entri'][j] = " ".join(new_entry)
            
            new_entry_font_size_pos = []
            new_entry_font_size_pos.extend(array_simbol_font_size_pos)
            new_entry_font_size_pos.extend(result['entry_font_size_pos'][j])
            result['entry_font_size_pos'][j] = new_entry_font_size_pos    
    
    for l in range(len(result['entry_font_size_pos'])):
        if result['entry_font_size_pos'][l] != []:
            result['posisi_entry'][l] = result['entry_font_size_pos'][l][0][-1]
        
    return result

In [221]:
def clean_entry_font_size_paranthesis(data):
    clean_data = []
    i = 0
    
    while i < len(data):
        txt = data[i][0]
        if re.match(r'^.*\[.*?\].*$',str(txt)): ## kasus ...[..]...
            clean = re.sub(r'\[.*?\]',"",txt)
            if clean == "":
                i += 1
            else:
                clean_data.append([clean.strip(),data[i][1],data[i][2],data[i][3]])
                i += 1
        elif re.match(r'^.*\[.*',str(txt)): ## kasus ...[...
            nxt = i+1
            if nxt > len(data)-1: # i di indeks terakhir
                clean_data.append(data[i])
                break
                
            nxt_txt = data[nxt][0]
            while not re.match(r'^.*\].*$',str(nxt_txt)): # mencari "...]...."
                nxt += 1
                if nxt > len(data)-1: break
                nxt_txt = data[nxt][0]
            
            if nxt > len(data)-1: # jika "....]..." tidak ditemukan
                for k in range(i,nxt):
                    clean_data.append(data[k])
                break
            else:
                ## append [ pertama
                clean = txt.split("[",1)[0]
                if clean != "":
                    clean_data.append([clean.strip(),data[i][1],data[i][2],data[i][3]])
                    
                ## append ] kedua
                clean_nxt = nxt_txt.split("]",1)[1]
                if clean_nxt != "":
                    clean_data.append([clean_nxt.strip(),data[nxt][1],data[nxt][2],data[i][3]])
                
                i = nxt+1
        else:
            clean_data.append(data[i])
            i += 1
    
    return clean_data


def clean_entry_font_size_slash(data):
    clean_data = []
    i = 0
    
    while i < len(data):
        txt = data[i][0]
        if re.match(r'^.*\/.*?\/.*$',str(txt)): ## kasus .../../...
            clean = re.sub(r'\/.*?\/',"",txt)
            if clean == "":
                i += 1
            else:
                clean_data.append([clean.strip(),data[i][1],data[i][2],data[i][3]])
                i += 1
        elif re.match(r'^.*\/.*',str(txt)): ## kasus .../...
            nxt = i+1
            if nxt > len(data)-1: # i di indeks terakhir
                clean_data.append(data[i])
                break
                
            nxt_txt = data[nxt][0]
            while not re.match(r'^.*\/.*$',str(nxt_txt)): # mencari ".../...."
                nxt += 1
                if nxt > len(data)-1: break
                nxt_txt = data[nxt][0]
            
            if nxt > len(data)-1: # jika "..../..." tidak ditemukan
                for k in range(i,nxt):
                    clean_data.append(data[k])
                break
            else:
                ## append / pertama
                clean = txt.split("/",1)[0]
                if clean != "":
                    clean_data.append([clean.strip(),data[i][1],data[i][2],data[i][3]])
                    
                ## append / kedua
                clean_nxt = nxt_txt.split("/",1)[1]
                if clean_nxt != "":
                    clean_data.append([clean_nxt.strip(),data[nxt][1],data[nxt][2],data[i][3]])
                
                i = nxt+1
        else:
            clean_data.append(data[i])
            i += 1
    
    return clean_data

In [222]:
def fix_page(pages):
    clean_page = []
    cnt = 1;
    
    for i in range(len(pages)):
        if i == 0:
            clean_page.append(cnt)
        else:
            if isinstance(pages[i], list):
                clean_page.append([cnt,cnt+1])
                cnt += 1
            else:
                if isinstance(pages[i-1], list):
                    clean_page.append(cnt)
                else:
                    if pages[i] == pages[i-1]:
                        clean_page.append(cnt)
                    else:
                        cnt += 1
                        clean_page.append(cnt)
    return clean_page

In [223]:
def categorize_prakategorial(entries):
    output = []
    
    for i in entries:
        if i == "" or len(i)==1:
            output.append(0)
        else:
            if re.match(r'.*\,$',str(i)): 
                output.append(1)
            elif is_contain_only_whitespaces(i[-2]) and (i[-1] in POS):
                output.append(1)
            else:
                output.append(0)
    return output

> Main Program

In [234]:
def build_corpus_one_entry_by_font_pos(data):
    # tahapan awal, pendekatan dengan font
    result = join_seperate_entry(data)
    kategorisasi = categorize_main_entry(result["posisi_entry"], result["page"], result["is_padanan_lema"])
    result = seperate_joined_entry(result, kategorisasi)
#   result = fix_cross_page_entry(result)
    
    clean_result = clean_entry(result)
    clean_result = seperate_prakategorial(result)
    clean_result["is_padanan_lema"] = categorize_prakategorial(clean_result["Entri"])
    return clean_result

### Main Program (JSON) ###

In [235]:
directory_CSV = "CSV One Entry JSON With Font Approach"
directory_hasil = "CSV One Entry JSON With Font + Posisi Approach"

for filename in tqdm(os.listdir(directory_CSV)):
    print("====" + filename + "====")
    kamus = pd.read_csv(directory_CSV + "/" + filename)
    kamus = kamus.dropna()
    kamus = kamus.reset_index(drop=True)
    entries = kamus["Entri"].values.tolist()

    entries_font_size_pos = []
    for i in kamus["entry_font_size_pos"].values.tolist():
        entries_font_size_pos.append(literal_eval(i))

    posisi_entry = []
    for i in kamus["posisi_entry"].values.tolist():
        posisi_entry.append(literal_eval(i))

    page = []
    for i in kamus["page"].values.tolist():
        if not isinstance(i,int):
            page.append(literal_eval(i))
        else:
            page.append(int(i))

    input_data = {
        "Entri":entries,
        "entry_font_size_pos":entries_font_size_pos,
        "posisi_entry":posisi_entry,
        "page":page,
        "is_padanan_lema":kamus["is_padanan_lema"].values.tolist()
    }
    
    CSV_res = build_corpus_one_entry_by_font_pos(input_data)

    result_csv = pd.DataFrame.from_dict(CSV_res)
    result_csv = result_csv[result_csv["Entri"] != ""]
    result_csv = result_csv.reset_index(drop=True)

    result_csv = result_csv.dropna()
    result_csv = result_csv.reset_index(drop=True)
    
    new_filename = os.path.splitext(filename)[0]
    result_csv.to_csv(directory_hasil + "/" + new_filename + "-posisi.csv",index=False)

 56%|██████████████████████████████████████████████▋                                     | 5/9 [00:00<00:00, 41.83it/s]

====41. Kamus Bahasa Indonesia-Bali A-K (1997)-page_58_59-hasil-ekstraksi-one_entry_from_JSON-font.csv====
====43. Kamus Bahasa Indonesia-Bahasa Minangkabau I (1994)-page_23_24-hasil-ekstraksi-one_entry_from_JSON-font.csv====
====54. Kamus Bahasa Indonesia Mentawai (1998)-page_23_24-hasil-ekstraksi-one_entry_from_JSON-font.csv====
====57. Kamus Bahasa Bugis-Indonesia (1977)-page_20_21-hasil-ekstraksi-one_entry_from_JSON-font.csv====
====61. Kamus Banjar-Indonesia (1977)-page_51_52-hasil-ekstraksi-one_entry_from_JSON-font.csv====
====68. Kamus Dwibahasa Bahasa Talaud - Bahasa Indonesia (2018)-page_19_20-hasil-ekstraksi-one_entry_from_JSON-font.csv====
====77. Kamus Samawa-Indonesia Edisi 2 (2017)-page_27_28-hasil-ekstraksi-one_entry_from_JSON-font.csv====
====89. Kamus Dwibahasa Bahasa Mooi-Bahasa Indonesia (2017)-page_17_18-hasil-ekstraksi-one_entry_from_JSON-font.csv====
====91. Kamus Simalungun - Indonesia (edisi kedua) (2015)-page_48_49-hasil-ekstraksi-one_entry_from_JSON-font.csv==

100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 41.35it/s]


### Main Program Full Kamus (JSON)

In [236]:
directory_CSV = "[Full] CSV One Entry JSON With Font Approach"
directory_hasil = "[Full] CSV One Entry JSON With Font + Posisi Approach"

for filename in tqdm(os.listdir(directory_CSV)):
    print("====" + filename + "====")
    kamus = pd.read_csv(directory_CSV + "/" + filename)
    kamus = kamus.dropna()
    kamus = kamus.reset_index(drop=True)
    entries = kamus["Entri"].values.tolist()

    entries_font_size_pos = []
    for i in kamus["entry_font_size_pos"].values.tolist():
        entries_font_size_pos.append(literal_eval(i))

    posisi_entry = []
    for i in kamus["posisi_entry"].values.tolist():
        posisi_entry.append(literal_eval(i))

    page = []
    for i in kamus["page"].values.tolist():
        if not isinstance(i,int):
            page.append(literal_eval(i))
        else:
            page.append(int(i))

    input_data = {
        "Entri":entries,
        "entry_font_size_pos":entries_font_size_pos,
        "posisi_entry":posisi_entry,
        "page":page,
        "is_padanan_lema":kamus["is_padanan_lema"].values.tolist()
    }
    
    CSV_res = build_corpus_one_entry_by_font_pos(input_data)

    result_csv = pd.DataFrame.from_dict(CSV_res)
    result_csv = result_csv[result_csv["Entri"] != ""]
    result_csv = result_csv.reset_index(drop=True)

    result_csv = result_csv.dropna()
    result_csv = result_csv.reset_index(drop=True)
    
    new_filename = os.path.splitext(filename)[0]
    result_csv.to_csv(directory_hasil + "/" + new_filename + "-posisi.csv",index=False)

  0%|                                                                                           | 0/48 [00:00<?, ?it/s]

====10. Kamus Bahasa Indonesia-Dayak Deah Edisi I (2013)-hasil-ekstraksi-one_entry_from_JSON-font.csv====


  2%|█▋                                                                                 | 1/48 [00:02<01:42,  2.19s/it]

====12. Kamus Bahasa Indonesia-Kaidipang L-Z (2000)-hasil-ekstraksi-one_entry_from_JSON-font.csv====


  4%|███▍                                                                               | 2/48 [00:06<02:41,  3.52s/it]

...

 96%|██████████████████████████████████████████████████████████████████████████████▌   | 46/48 [01:56<00:04,  2.14s/it]

====9. Kamus Manado-Indonesia (1985)-hasil-ekstraksi-one_entry_from_JSON-font.csv====


 98%|████████████████████████████████████████████████████████████████████████████████▎ | 47/48 [01:58<00:01,  1.94s/it]

====91. Kamus Simalungun - Indonesia (edisi kedua) (2015)-hasil-ekstraksi-one_entry_from_JSON-font.csv====


100%|██████████████████████████████████████████████████████████████████████████████████| 48/48 [02:01<00:00,  2.54s/it]


### Main Program (XML) ###

In [17]:
directory_CSV = "CSV XML All Information"
directory_hasil = "CSV One Entry XML With Font + Posisi Approach"

for filename in tqdm(os.listdir(directory_CSV)):
    data = pd.read_csv(directory_CSV + "/" + filename)
    data = data.dropna()
    data = data.reset_index(drop=True)
    
    input_texts = data["kata"].values.tolist()
    input_fonts = data["font"].values.tolist()
    
    for i in input_texts:
        i = str(i)
    
    # masih harus diperbaiki
    list_x = data["x"].values.tolist()
    list_y = data["y"].values.tolist()
    input_posisi = []
    
    i = 0
    while i < len(list_x):
        input_posisi.append((list_x[i],list_y[i],list_x[i],list_y[i])) # masih harus diperbaiki
        i += 1
        
    input_pages = data["page"].values.tolist()
    new_filename = os.path.splitext(filename)[0]
    
    if is_contain_bold_and_italic(input_fonts):
        print("====" + new_filename + "====")
        try:
            entry, entry_with_font, entry_with_posisi, posisi_entry, page_entry = build_corpus_one_entry_by_font_from_XML(
                input_texts, 
                input_fonts, 
                input_posisi, 
                input_pages
            )
            CSV_res = {
                "Entri":entry,
                "Entri with Font":entry_with_font,
                "Entri with Posisi":entry_with_posisi,
                "Posisi":posisi_entry,
                "Page": page_entry
            }

            result_csv = pd.DataFrame.from_dict(CSV_res)
            result_csv.to_csv(directory_hasil + "/" + new_filename + "-one_entry_from_XML_font_posisi.csv",index=False)
        except:
            print("==== Kamus Gagal ====")
            print(new_filename)

 83%|████████████████████████████████████████████████████████████████████▎             | 15/18 [00:00<00:00, 62.44it/s]

====41. Kamus Bahasa Indonesia-Bali A-K (1997)-page_58_59_kata====
====43. Kamus Bahasa Indonesia-Bahasa Minangkabau I (1994)-page_23_24_kata====
====45. Kamus Bahasa Indonesia-Bahasa Gayo II (1996)-page_56_57_kata====
====54. Kamus Bahasa Indonesia Mentawai (1998)-page_23_24_kata====
====57. Kamus Bahasa Bugis-Indonesia (1977)-page_20_21_kata====
====61. Kamus Banjar-Indonesia (1977)-page_51_52_kata====
====68. Kamus Dwibahasa Bahasa Talaud - Bahasa Indonesia (2018)-page_19_20_kata====
====77. Kamus Samawa-Indonesia Edisi 2 (2017)-page_27_28_kata====


100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 60.71it/s]

====89. Kamus Dwibahasa Bahasa Mooi-Bahasa Indonesia (2017)-page_17_18_kata====
====91. Kamus Simalungun - Indonesia (edisi kedua) (2015)-page_48_49_kata====


### Cek Kamus ###

In [13]:
target = "CSV One Entry JSON With Font + Posisi Approach/"
kamus = pd.read_csv(target + "43. Kamus Bahasa Indonesia-Bahasa Minangkabau I (1994)-page_23_24-hasil-ekstraksi-one_entry_from_JSON-font-posisi.csv")

In [14]:
# tampilkan seluruh baris dan seluruh nilai pada kolom
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

display(kamus)

# reset option
pd.reset_option("display")

Entri  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              23   
1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           anutan n 1 conto elok; ikutan; tuladan: guru hendaknya dapat menjadi -- murid-muridnya gum handaknyo dapek manjadi conto elok dek murik-muriknyo; 2 kayakinan diikuik   
2                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       a.nyam. v   
3                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               menganyam v maanyam: dia pandai == daun kelapa inyo pandai maanyam daun karambia;   
4                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            anyaman n dianyam; anyaman = =nya rapi sekali anyamannyo rancak bana   
5                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      a.pa.bi.Ia p apobilo; bib: --Tuhan mengizinkan tahun depan saya in gin naik haji apobilo Tuhan maizinan, taun muko ambo taragak naiak haji   
6                                                                                                                        